In [1]:
import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
from tvtk.api import tvtk, write_data
from nf2.evaluation.unpack import load_cube

In [3]:
GD = Path('/Users/mgjeon/Library/CloudStorage/GoogleDrive-A/My Drive/Notes/Work/PINN/')
NAS = Path('/Volumes/drive/Research/SSWLAB/NF2')

# NOAA 11158

In [8]:
nf2_file = GD / Path('ar_series_377_2011-02-12T00:00:00/base/20110215_000000_TAI/extrapolation_result.nf2')

In [9]:
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
state = torch.load(nf2_file, map_location=device)
model = torch.nn.DataParallel(state['model'])
cube_shape = state['cube_shape']
z = cube_shape[2]
coords = np.stack(np.mgrid[:cube_shape[0]:1, :cube_shape[1]:1, :z:1], -1)
spatial_norm = 160
coords = torch.tensor(coords / spatial_norm, dtype=torch.float32)
coords_shape = coords.shape

ModuleNotFoundError: No module named 'nf2'

In [41]:
#state

In [ ]:
cube = []
batch_size = 1000
coords = coords.view((-1, 3))
it = range(int(np.ceil(coords.shape[0] / batch_size)))
for k in tqdm(it):
    coord = coords[k * batch_size: (k + 1) * batch_size]
    coord = coord.to(device)
    coord.requires_grad = True
    cube += [model(coord).detach().cpu()]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8715/8715 [00:48<00:00, 180.66it/s]


In [16]:
cube = torch.cat(cube)
cube = cube.view(*coords_shape).numpy()
b_norm = 2500
b = cube * b_norm

In [20]:
vtk_path = str(NAS / 'NOAA11158/11158_20110215_000000_colab.vtk')

In [21]:
bin = 2
Mm_per_pix = 360e-3 * bin
# Unpack
dim = b.shape[:-1]
# Generate the grid
pts = np.stack(np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]], -1).astype(np.int64) * Mm_per_pix
# reorder the points and vectors in agreement with VTK
# requirement of x first, y next and z last.
pts = pts.transpose(2, 1, 0, 3)
pts = pts.reshape((-1, 3))
vectors = b.transpose(2, 1, 0, 3)
vectors = vectors.reshape((-1, 3))

sg = tvtk.StructuredGrid(dimensions=dim, points=pts)
sg.point_data.vectors = vectors
sg.point_data.vectors.name = 'B'
write_data(sg, vtk_path)

# NOAA 12673

In [44]:
nf2_file = NAS / Path('NF2_DB/author/example_7115.nf2')

In [45]:
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
state = torch.load(nf2_file, map_location=device)
model = torch.nn.DataParallel(state['model'])
cube_shape = state['cube_shape']
z = cube_shape[2]
coords = np.stack(np.mgrid[:cube_shape[0]:1, :cube_shape[1]:1, :z:1], -1)
spatial_norm = 160
coords = torch.tensor(coords / spatial_norm, dtype=torch.float32)
coords_shape = coords.shape

In [42]:
#state

In [46]:
cube = []
batch_size = 1000
coords = coords.view((-1, 3))
it = range(int(np.ceil(coords.shape[0] / batch_size)))
for k in tqdm(it):
    coord = coords[k * batch_size: (k + 1) * batch_size]
    coord = coord.to(device)
    coord.requires_grad = True
    cube += [model(coord).detach().cpu()]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12329/12329 [01:08<00:00, 180.01it/s]


In [47]:
cube = torch.cat(cube)
cube = cube.view(*coords_shape).numpy()
b_norm = 2500
b = cube * b_norm

In [48]:
vtk_path = str(NAS / 'NOAA12673/12673_20170906_083600_author.vtk')

In [49]:
bin = 2
Mm_per_pix = 360e-3 * bin
# Unpack
dim = b.shape[:-1]
# Generate the grid
pts = np.stack(np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]], -1).astype(np.int64) * Mm_per_pix
# reorder the points and vectors in agreement with VTK
# requirement of x first, y next and z last.
pts = pts.transpose(2, 1, 0, 3)
pts = pts.reshape((-1, 3))
vectors = b.transpose(2, 1, 0, 3)
vectors = vectors.reshape((-1, 3))

sg = tvtk.StructuredGrid(dimensions=dim, points=pts)
sg.point_data.vectors = vectors
sg.point_data.vectors.name = 'B'
write_data(sg, vtk_path)